## Web Scraping from data source

In [1]:
# dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# report generator url
url = "http://www9.health.gov.au/cda/source/rpt_2_sel.cfm"

In [3]:
# use splinter to navigate to page
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\fernb\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [4]:
# set query ranges and empty lists for data return
queryyears = [2015,2016,2017,2018,2019,2020]
diseaselist =[]
diseasedflist=[]
diseasegroupings = []
eachgrouplist = []
summarytable = pd.DataFrame(columns = ["Disease_Name","Infection_Rate","Location","Disease_Group","Year"])

In [5]:
# loop through years for field options
for year in queryyears:
    
    # use splinter to select and submit form based on field options
    browser.find_by_id('report_option_5').first.click()
    browser.find_by_id('sel_year').first.select(str(year))
    browser.find_by_id('CTIME1').click() 
    
    #get html table data into dataframe using pandas
    tables = pd.read_html(browser.html)
    df = tables[0]
    dropped = df.dropna(axis=0)
    
    #form initial dataframes on first year
    if year == 2015:
        print("first")
        #get list of disease groups
        diseasegroups = dropped.sort_values(by=['ACT'],ascending=False)[:8]
        diseasegroups = diseasegroups["ACT"].sort_index()
        indexlist = diseasegroups.index
        diseaselist = diseasegroups.tolist()
        diseasedflist = diseasegroups.tolist()        

        # loop through dataframe and slice based on disease groupings
        for i in range(len(indexlist)):
            if i !=7:
                dlist = dropped[indexlist[i]:indexlist[i+1]-1]["Unnamed: 0"]
            else:
                dlist = dropped[indexlist[i]:]["Unnamed: 0"]
            
            # get group and diseases paired for later database load
            dlist = dlist.values.tolist()
            for d in dlist:
                diseasegroupings.append(diseaselist[i])
                eachgrouplist.append(d)
#             dlist.append('Location')
#             dlist.append('Year')
#             diseasedflist[i] 
            
        
        #slice and append data to dataframe per disease grouping
        for i in range(len(diseaselist)):
            if i !=7:
                item = dropped[indexlist[i]:indexlist[i+1]-1]
            else:
                item = dropped[indexlist[i]:]
            
            locationlist = ["ACT","NSW","NT","QLD","SA","TAS","VIC","WA","Aust","Last 5yearsmean"]
            for location in locationlist:
                state = item[["Unnamed: 0",location]]
                state["Location"] = [location for x in range(len(state))]
                state["Disease_Group"] = [diseaselist[i] for x in range(len(state))]
                state["Year"] = [year for x in range(len(state))]
                state.rename(columns = {'Unnamed: 0':"Disease_Name", location:"Infection_Rate"}, inplace = True)
                summarytable = summarytable.append(state, ignore_index=True)
            
            
#             item.set_index('Unnamed: 0', inplace=True)
#             item = item[["ACT","NSW","NT","QLD","SA","TAS","VIC","WA","Aust","Last 5yearsmean"]].T
#             item["Year"] = [year for x in range(10)]
#             item.reset_index(inplace =True)
#             item.rename(columns = {'index':"Location"}, inplace = True)
#             diseasedflist[i]=diseasedflist[i].append(item, ignore_index=True)  
        

    #loop through rest of years to append to dataframes
    else:
        
        
        for i in range(len(diseaselist)):
            if i !=7:
                item = dropped[indexlist[i]:indexlist[i+1]-1]
            else:
                item = dropped[indexlist[i]:]
                
            locationlist = ["ACT","NSW","NT","QLD","SA","TAS","VIC","WA","Aust","Last 5yearsmean"]
            for location in locationlist:
                state = item[["Unnamed: 0",location]]
                state["Location"] = [location for x in range(len(state))]
                state["Disease_Group"] = [diseaselist[i] for x in range(len(state))]
                state["Year"] = [year for x in range(len(state))]
                state.rename(columns = {'Unnamed: 0':"Disease_Name", location:"Infection_Rate"}, inplace = True)
                summarytable = summarytable.append(state, ignore_index=True)
#             item.set_index('Unnamed: 0', inplace=True)
#             item = item[["ACT","NSW","NT","QLD","SA","TAS","VIC","WA","Aust","Last 5yearsmean"]].T
#             item["Year"] = [year for x in range(10)]
#             item.reset_index(inplace =True)
#             item.rename(columns = {'index':"Location"}, inplace = True)
#             diseasedflist[i]=diseasedflist[i].append(item, ignore_index=True)
    
    # close year report so loop can run another report
    browser.find_by_id('close').click() 

        
        

first


<ipython-input-5-61d70a16142e>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state["Location"] = [location for x in range(len(state))]
<ipython-input-5-61d70a16142e>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state["Disease_Group"] = [diseaselist[i] for x in range(len(state))]
<ipython-input-5-61d70a16142e>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [6]:
summarytable

,Disease_Name,Infection_Rate,Location,Disease_Group,Year
0,Hepatitis (NEC),0.0,ACT,Bloodborne diseases,2015
1,Hepatitis B (newly acquired),0.0,ACT,Bloodborne diseases,2015
2,Hepatitis B (unspecified),21.0,ACT,Bloodborne diseases,2015
3,Hepatitis C (newly acquired),3.5,ACT,Bloodborne diseases,2015
4,Hepatitis C (unspecified),44.2,ACT,Bloodborne diseases,2015
...,...,...,...,...,...
4195,Leptospirosis,0.5,Last 5yearsmean,Zoonoses,2020
4196,Lyssavirus infection (NEC),0.0,Last 5yearsmean,Zoonoses,2020
4197,Ornithosis,0.1,Last 5yearsmean,Zoonoses,2020
4198,Q fever,2.2,Last 5yearsmean,Zoonoses,2020


In [7]:
## create dictionary of disease groupings

dic = {"DiseaseGroup":diseasegroupings,"Disease":eachgrouplist}

groupeddf = pd.DataFrame.from_dict(dic)

In [8]:
groupeddf

,DiseaseGroup,Disease
0,Bloodborne diseases,Hepatitis (NEC)
1,Bloodborne diseases,Hepatitis B (newly acquired)
2,Bloodborne diseases,Hepatitis B (unspecified)
3,Bloodborne diseases,Hepatitis C (newly acquired)
4,Bloodborne diseases,Hepatitis C (unspecified)
...,...,...
65,Zoonoses,Leptospirosis
66,Zoonoses,Lyssavirus infection (NEC)
67,Zoonoses,Ornithosis
68,Zoonoses,Q fever


In [9]:
browser.quit()

In [10]:
## create sqlite database

from sqlalchemy import create_engine

path = f"sqlite:///diseases.sqlite"
engine = create_engine(path)

connection = engine.connect()



In [11]:
summarytable.to_sql("DiseaseSummary",connection, if_exists='replace',index=True)

In [ ]:
## add disease groups to database
groupeddf.to_sql("DiseaseGroups",connection, if_exists='replace',index=True)



In [12]:
## add primary key 

connection.execute("PRAGMA foreign_keys=off;")

connection.execute("BEGIN TRANSACTION;")

connection.execute("ALTER TABLE DiseaseSummary RENAME TO old_table;")

connection.execute("CREATE TABLE DiseaseSummary (id INTEGER NOT NULL PRIMARY KEY, 'Disease_Name' TEXT NOT NULL, 'Infection_Rate' REAL NOT NULL, 'Location' TEXT NOT NULL, 'Disease_Group' TEXT NOT NULL, Year INTEGER NOT NULL);")

connection.execute("INSERT INTO DiseaseSummary SELECT * FROM old_table;")

connection.execute("DROP TABLE old_table;")

connection.execute("PRAGMA foreign_keys=on;")

OperationalError: (sqlite3.OperationalError) cannot commit - no transaction is active
[SQL: COMMIT;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [ ]:
# connection.execute("DROP TABLE Bloodbornediseases;")